In [ ]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

In [ ]:
from simple_vector_db.quantization.vector_quantizer import VectorQuantizer
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_digits
import pandas as pd
import psutil
import numpy as np
import sys
import seaborn as sns
import dataget
import random

In [ ]:
def load_fashion_mnist(n_sample=1000):
    X_train, y_train, X_test, y_test = dataget.image.fashion_mnist().get()
    X = np.vstack([X_train, X_test])
    images = []
    for image in X:
        images.append(image.flatten())
    random.shuffle(images)
    images = images[0:n_sample]
    return images

In [ ]:

import numpy as np

def taille_memoire(entier):
    if np.iinfo(np.int8).min <= entier <= np.iinfo(np.int8).max:
        return np.dtype(np.int8).itemsize
    elif np.iinfo(np.int16).min <= entier <= np.iinfo(np.int16).max:
        return np.dtype(np.int16).itemsize
    elif np.iinfo(np.int32).min <= entier <= np.iinfo(np.int32).max:
        return np.dtype(np.int32).itemsize
    elif np.iinfo(np.int64).min <= entier <= np.iinfo(np.int64).max:
        return np.dtype(np.int64).itemsize
    else:
        return None


taille_memoire(9000000)

In [ ]:
vectors_to_quantize = load_digits().data
#vectors_to_quantize = load_fashion_mnist()

In [ ]:
m_chunks_list = list(range(4, 64,4))
res = []
for m_chunk in [2]+m_chunks_list:
    k_centroids_list = np.array(range(10, 500,25))
    for k_centroids in k_centroids_list:
        try:
            quantizer = VectorQuantizer(m_chunks=m_chunk, nb_subspace_centroids=k_centroids)
            quantized_vectors = quantizer.quantize_vectors(vectors_to_quantize)
            #print(m_chunk, k_centroids, (vectors_to_quantize.size * vectors_to_quantize.itemsize) / 1024,
            #      (quantized_vectors.size * quantized_vectors.itemsize) / 1024)
            rebuilt_vector = quantizer.rebuild_vector(quantized_vectors[0])
            compression_mse = mean_squared_error(vectors_to_quantize[0], rebuilt_vector)
            res.append(
                {"Nombre de sections": m_chunk, "Nombre de centroïdes par section": k_centroids,
                 "mesure MSE": compression_mse,
                 "mémoire en kB": (len(quantized_vectors) * m_chunk * taille_memoire(k_centroids)) / 1024})
        except ValueError:
            print(m_chunk, k_centroids, "ERROR")
        res_df = pd.DataFrame(res)
        res_df.to_csv("../figures/quantization_mse_vs_params_FMNIST.csv")


In [ ]:
colors = ["#EE0023", "#0E2356", "#00AD00", "#00A3BE", "#00A795", "#FFE300", "#FF6200", "#9A78F0", "#9FA7BB", "#F899A7"]
custom_palette = sns.color_palette(colors)

In [ ]:
sns_plot = sns.scatterplot(data=res_df, x="Nombre de centroïdes par section", y="mesure MSE", hue="Nombre de sections",
                           palette=custom_palette)
fig = sns_plot.get_figure()
fig.savefig("../figures/mse_vs_params.png")

In [ ]:
sns_plot = sns.scatterplot(data=res_df, x="Nombre de centroïdes par section", y="mémoire en kB", hue="Nombre de sections",
                           palette=custom_palette)
fig = sns_plot.get_figure()
fig.savefig("../figures/memory_vs_params.png")

## PQ parameters vs Recall 

In [ ]:
bench_kmeans = pd.read_csv("../figures/bench_results_pq.csv")
sns_plot = sns.scatterplot(data=bench_kmeans, y="Nombre de centroïdes par section", x="rappel @ 10",
                           hue="Nombre de sections",
                           palette=custom_palette)
fig = sns_plot.get_figure()
fig.savefig("../figures/pq_recall_vs_params.png")

## PQ RECALL VS SPEED

In [ ]:
bench_kmeans.groupby("rappel @ 10").agg({"nb_requests_per_second": "mean"})

In [ ]:

sns_plot = sns.scatterplot(data=bench_kmeans, x="rappel @ 10", y="Nombre de requêtes par seconde",
                           palette=custom_palette, hue="Nombre de sections")
fig = sns_plot.get_figure()
fig.savefig("../figures/pq_recall_vs_speed.png")

In [ ]:

sns_plot = sns.scatterplot(data=bench_kmeans, x="rappel @ 10", y="Nombre de requêtes par seconde",
                           palette=custom_palette, hue="Nombre de centroïdes par section")
fig = sns_plot.get_figure()
fig.savefig("../figures/pq_recall_vs_speed.png")

## IVKMEANS parameters vs Recall 

In [ ]:
bench_kmeans = pd.read_csv("../figures/bench_results_kmeans.csv")
sns_plot = sns.scatterplot(data=bench_kmeans, y="Nombre de clusters", x="Rappel @ 10",
                           hue="Valeur de n_probes",
                           palette=custom_palette)
fig = sns_plot.get_figure()
fig.savefig("../figures/kmeans_recall_vs_params.png")

In [ ]:
sns_plot = sns.scatterplot(data=bench_kmeans, x="Rappel @ 10", y="Nombre de requêtes par seconde",
                           palette=custom_palette, hue="Valeur de n_probes")
fig = sns_plot.get_figure()
fig.savefig("../figures/kmeans_recall_vs_speed_clusters_nprobs.png")

In [ ]:
bench_kmeans['Intervalle du nombre de clusters '] = pd.cut(bench_kmeans["Nombre de clusters"], bins=5, precision=0)

In [ ]:
sns_plot = sns.scatterplot(data=bench_kmeans, x="Rappel @ 10", y="Nombre de requêtes par seconde",
                           palette=custom_palette, hue='Intervalle du nombre de clusters ')
fig = sns_plot.get_figure()
fig.savefig("../figures/kmeans_recall_vs_speed_clusters.png")

In [ ]:

sns_plot = sns.scatterplot(data=bench_kmeans, x="Rappel @ 10", y="Nombre de requêtes par seconde",
                           palette=custom_palette, hue='Intervalle du nombre de clusters ')
fig = sns_plot.get_figure()
fig.savefig("../figures/kmeans_recall_vs_speed_clusters.png")